# Stroke Classfier
> - Gabrile Serafini
> - Marco Sousa-Poza

This Notebook discribes the computations made to find an optimal stroke classifier based on real healthcare data.

## Exploring the Data
In the following the data will be read and prepared for further use.

In [8]:
# All the necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
# Reading the data into a data frame
df_health = pd.read_csv('data/healthcare-dataset-stroke-data.csv')
df_health.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


We see that the data contains some NaN values. If there are not too many we can delete them.

In [10]:
print("The number of NaN values for each column is:")
print(df_health.isna().sum())
print("There are {} rows in this df.".format(df_health.shape[0]))

The number of NaN values for each column is:
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64
There are 5110 rows in this df.


Since there are not that many NaN values we can drop them. We will also drop the id as it is not important.

In [11]:
df_health.dropna(axis=1, inplace=True)
df_health.drop('id', axis=1, inplace=True)

Next we change the nominal attributes to numbers. This is important for the more mathematical models we are going to use later on.

In [12]:
obj_df = df_health.select_dtypes(include=['object'])
for col in obj_df.columns:
    print(obj_df[col].value_counts())

Female    2994
Male      2115
Other        1
Name: gender, dtype: int64
Yes    3353
No     1757
Name: ever_married, dtype: int64
Private          2925
Self-employed     819
children          687
Govt_job          657
Never_worked       22
Name: work_type, dtype: int64
Urban    2596
Rural    2514
Name: Residence_type, dtype: int64
never smoked       1892
Unknown            1544
formerly smoked     885
smokes              789
Name: smoking_status, dtype: int64


In [19]:
# We again find some hidden NaN values. We can delete those:
df_health.drop(df_health[df_health['gender'] == 'Other'].index, inplace=True)
# NOTE: there are quite many instances where it is unknown if they smoke or not. We will discide later.
gender_dict = {'Female':1, 'Male':0}
married_dict = {'Yes':1, 'No':0}
worktype_dict = {'Private':4, 'Self-employed':3, 'children':2, 'Govt_job':1, 'Never_worked':0}
residence_dict = {'Urban':1, 'Rural':0}
smoke_dict = {'never smoked':3,'Unknown':2,'formerly smoked':1,'smokes':0}
df_health_numerical = df_health.replace(
    {
        'gender':gender_dict,
        'ever_married':married_dict,
        'work_type':worktype_dict,
        'Residence_type':residence_dict,
        'smoking_status':smoke_dict
    }
)
print(df_health_numerical.dtypes)

gender                 int64
age                  float64
hypertension           int64
heart_disease          int64
ever_married           int64
work_type              int64
Residence_type         int64
avg_glucose_level    float64
smoking_status         int64
stroke                 int64
dtype: object
